In [4]:
import sys
import numpy as np

#import matplotlib.pyplot as plt
#import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import sklearn.linear_model
import sklearn.metrics

from scipy.stats import spearmanr as cor

from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Merge
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from keras.layers import Concatenate
from keras.models import load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM
from sklearn import preprocessing

Using TensorFlow backend.


In [35]:
def makeDNN(in_size, dropout=0.2):
    l2_lam = 5e-07 
    l1_lam = 1e-08 
    model = Sequential()
    model.add(Flatten(input_shape=(in_size)))
    model.add(Dense(150, kernel_regularizer=regularizers.l1(l1_lam)))
    model.add(Activation("relu"))
    model.add(Dropout(dropout))
    model.add(Dense(100, kernel_regularizer=regularizers.l1(l1_lam))) 
    model.add(Activation("relu"))
    model.add(Dropout(dropout))
    model.add(Dense(50, kernel_regularizer=regularizers.l1(l1_lam))) 
    model.add(Activation("relu"))
    model.add(Dense(3, kernel_regularizer=regularizers.l1(l1_lam)))
    model.add(Activation("softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def makeCNN(in_size, dropout=0.2):
    nkernels = [80,120,160]
    l2_lam = 5e-07 
    l1_lam = 1e-08 
    # isize = (1,300,35*3)
    model = Sequential()
    model.add(Conv2D(nkernels[0], kernel_size=(1,8), strides=(1,1), padding='same', input_shape=in_size, kernel_regularizer=regularizers.l2(l2_lam)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(1,4), strides=(1,4)))
    model.add(Dropout(0.2))

    model.add(Conv2D(nkernels[1], kernel_size=(1,8), strides=(1,1), padding='same', kernel_regularizer=regularizers.l2(l2_lam)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(1,4), strides=(1,4)))
    model.add(Dropout(0.2))

    model.add(Conv2D(nkernels[2], kernel_size=(1,8), strides=(1,1), padding='same', kernel_regularizer=regularizers.l2(l2_lam)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(50, kernel_regularizer=regularizers.l1(l1_lam)))
    model.add(Activation('relu'))
    model.add(Dense(3, kernel_regularizer=regularizers.l1(l1_lam)))
    model.add(Activation("softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [29]:
suffix = "-2017_01_01-2017_07_03.npy"
X1 = np.load("X"+suffix)
X2 = np.load("X2"+suffix)
X3 = np.load("X3"+suffix)
Y = np.load("Y-2017_01_01-2017_07_03.npy")

In [32]:
X = np.concatenate([X1,X2,X3], axis=1)
model1 = makeDNN(X.shape[1:])
model1.fit(X[:128], Y[:128],
                  epochs=ep, 
                  batch_size=64,
                  callbacks=callbacks_list,
                  validation_split=0.2)

Epoch 1/10
128/128 [==============================] - 2s - loss: 2.0751 - acc: 0.4922       
Epoch 2/10
128/128 [==============================] - 0s - loss: 1.4266 - acc: 0.7891     
Epoch 3/10
128/128 [==============================] - 0s - loss: 1.6905 - acc: 0.7109     
Epoch 4/10
128/128 [==============================] - 0s - loss: 1.5339 - acc: 0.7969     
Epoch 5/10
128/128 [==============================] - 0s - loss: 1.7982 - acc: 0.7344     
Epoch 6/10
128/128 [==============================] - 0s - loss: 1.3683 - acc: 0.7969     
Epoch 7/10
128/128 [==============================] - 0s - loss: 1.2017 - acc: 0.7500     
Epoch 8/10
128/128 [==============================] - 0s - loss: 1.9198 - acc: 0.7109     
Epoch 9/10
128/128 [==============================] - 0s - loss: 1.3819 - acc: 0.7812     
Epoch 10/10
128/128 [==============================] - 0s - loss: 1.1051 - acc: 0.7344     


In [36]:
X = np.concatenate([X1,X2,X3], axis=1)
X = np.expand_dims(X, axis=1)
model2 = makeCNN(X.shape[1:])
model2.fit(X[:128], Y[:128])

Epoch 1/10
128/128 [==============================] - 4s - loss: 1.0196 - acc: 0.6016     
Epoch 2/10
128/128 [==============================] - 2s - loss: 0.6018 - acc: 0.8281     
Epoch 3/10
128/128 [==============================] - 2s - loss: 0.3999 - acc: 0.8359     
Epoch 4/10
128/128 [==============================] - 2s - loss: 0.3505 - acc: 0.8359     
Epoch 5/10
128/128 [==============================] - 2s - loss: 0.4063 - acc: 0.8281     
Epoch 6/10
128/128 [==============================] - 2s - loss: 0.2852 - acc: 0.8750     
Epoch 7/10
128/128 [==============================] - 2s - loss: 0.3203 - acc: 0.8672     
Epoch 8/10
128/128 [==============================] - 2s - loss: 0.2563 - acc: 0.9062     
Epoch 9/10
128/128 [==============================] - 2s - loss: 0.1773 - acc: 0.9453     
Epoch 10/10
128/128 [==============================] - 2s - loss: 0.1669 - acc: 0.9453     
